In [1]:
import numpy as np
import librosa
from augment import *

In [2]:
X_tr = np.load('../gtzan_tr.npy')

In [3]:
SAMPLE_RATE = 22050
X_noised_tr_sample = augment_audio(X_tr[0: 5], SAMPLE_RATE)

In [4]:
import IPython.display as ipd
ipd.Audio(X_noised_tr_sample[0], rate=SAMPLE_RATE) # load a NumPy array
